## Dependencies

In [11]:
import pandas as pd

## Data Handling

In [12]:
dfMouseMetadata=pd.read_csv("Resources/Mouse_metadata.csv")   
dfStudyResult=pd.read_csv("Resources/Study_results.csv")

study_data = pd.merge(dfMouseMetadata, dfStudyResult, how="left", on="Mouse ID")
study_data.head()

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
0,k403,Ramicane,Male,21,16,0,45.000000,0
1,k403,Ramicane,Male,21,16,5,38.825898,0
2,k403,Ramicane,Male,21,16,10,35.014271,1
3,k403,Ramicane,Male,21,16,15,34.223992,1
4,k403,Ramicane,Male,21,16,20,32.997729,1


## Data Cleaning

In [13]:
study_data=study_data.rename(columns={
    "Mouse ID":"Mouse_ID",
    "Drug Regimen":"Drug_Regimen",
    "Age_months":"Age_Months",
    "Weight (g)":"Weight",
    "Tumor Volume (mm3)":"Tumor_Volume",
    "Metastatic Sites":"Metastatic_Sites"
})
study_data.head()

,Mouse_ID,Drug_Regimen,Sex,Age_Months,Weight,Timepoint,Tumor_Volume,Metastatic_Sites
0,k403,Ramicane,Male,21,16,0,45.000000,0
1,k403,Ramicane,Male,21,16,5,38.825898,0
2,k403,Ramicane,Male,21,16,10,35.014271,1
3,k403,Ramicane,Male,21,16,15,34.223992,1
4,k403,Ramicane,Male,21,16,20,32.997729,1


In [14]:
# checking the number of mice

len(study_data["Mouse_ID"].unique())

249

In [15]:
# Getting the duplicate mice by ID number that shows up for Mouse ID and Timepoint. 

dfDuplicatedMouseIDs=study_data.loc[study_data.duplicated(subset=['Mouse_ID', 'Timepoint']),'Mouse_ID'].unique()
dfDuplicatedMouseIDs

array(['g989'], dtype=object)

In [16]:
# Showing the belongs to duplicated mice ID

dfDuplicatedMouseID=study_data.loc[study_data["Mouse_ID"]=="g989"]
dfDuplicatedMouseID

,Mouse_ID,Drug_Regimen,Sex,Age_Months,Weight,Timepoint,Tumor_Volume,Metastatic_Sites
908,g989,Propriva,Female,21,26,0,45.000000,0
909,g989,Propriva,Female,21,26,0,45.000000,0
910,g989,Propriva,Female,21,26,5,48.786801,0
911,g989,Propriva,Female,21,26,5,47.570392,0
912,g989,Propriva,Female,21,26,10,51.745156,0
913,g989,Propriva,Female,21,26,10,49.880528,0
914,g989,Propriva,Female,21,26,15,51.325852,1
915,g989,Propriva,Female,21,26,15,53.442020,0
916,g989,Propriva,Female,21,26,20,55.326122,1
917,g989,Propriva,Female,21,26,20,54.657650,1


In [17]:
# deleting the data belongs to duplicated mice id 

dfCleanedData=study_data[study_data["Mouse_ID"].isin(dfDuplicatedMouseIDs)==False]

len(dfCleanedData["Mouse_ID"].unique())

248